In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-gemini/genai-processors/blob/main/notebooks/agent_gemma.ipynb)

# Function Calling with Transformers 🤖

The GenAI Processor library provides an easy way to call a transformer with a
Function Calling loop. This notebook provides a step-by-step guide on how to do
this.



## 1. 🛠️ Setup

First, install the GenAI Processors library:

In [ ]:
!pip install genai-processors

## 2. 👆 Define the functions to be called

The tool definition is automatically derived from the function signature.

A good docstring covering the arguments and returned value is therefore advised.
Although introspection allows the model to see the function schema even without
a docstring, its absence will still hinder the model's ability to generate a
correct function call.

In [ ]:
def get_temperature(location: str) -> str:
  """Gets the temperature in Celsius at location including a weather description.

  Args:
    location: name of the city, region or place where the weather is requested.
  """
  return {'temperature': 21, 'weather': 'rainy'}


def to_fahrenheit(celsius: float) -> float:
  """Gets the temperature in Fahrenheit from a temperature in Celsius.

  Args:
    celsius: temperature in Celsius.
  """
  fahrenheit = (celsius * 9 / 5) + 32
  return fahrenheit

## 3. ✨ Create the Transformer model

Set the following parameters in the form below:

-   **model_name**: Set the model name to a HuggingFace transformer name. The
    model should support tool use. Note that loading the model can take a while.

-   **log_chat_template**: True if you want to log the prompt to the transformer
    in your colab runtime logs. This is quite verbose but useful for debugging.

-   **tool_response_format_dict**: True if the function declaration uses a
    dictionary instead of a plain string to represents its output. Set to False
    for std transformers.

-   **system_instructions**: this will be prepended to the transformer chat
    prompt.

In [ ]:
from genai_processors.core import transformers_model

model_name = ''  # @param {type: "string"}
tool_response_format_dict = True  # @param {"type":"boolean"}
log_chat_template = True  # @param {"type":"boolean"}
system_instruction = ''  # @param {type: "string"}

tool_response_format = 'dict' if tool_response_format_dict else 'string'


# Initialize the GenAI model processor
hf_model = transformers_model.TransformersModel(
    model_name=model_name,
    generate_content_config=transformers_model.GenerateContentConfig(
        system_instruction=system_instruction,
        tools=[get_temperature, to_fahrenheit],
    ),
    log_chat_template=log_chat_template,
    tool_response_format=tool_response_format,  # NOTE: Set to True for std models.
)

Then adding function calling is done as follows:

In [ ]:
# @title Adding function calling loop

from genai_processors import debug
from genai_processors.core import function_calling

fc = function_calling.FunctionCalling(
    debug.print_stream('MODEL') + hf_model,
    fns=[get_temperature, to_fahrenheit],
)

## 4. ▶️ Run the function calling processor

The function calling processor is typically used on a stream of user prompts.

In [ ]:
from genai_processors import streams
import nest_asyncio

nest_asyncio.apply()  # Needed to run async loops in Colab

input_stream = streams.stream_content(['What is the temperature in London?'])

async for part in fc(input_stream):
  if not part.substream_name:
    # default substream - contains what the user would see.
    print(f'{part.text}', flush=True, end='')

  if part.substream_name:
    # subtream_name = "function_call" / internal function calls.
    if part.function_call:
      print(
          f'\033[96m FC: {part.function_call.name}:'
          f' {part.function_call.args}\033[0m ',
          flush=True,
      )
    elif part.function_response:
      print(
          f'\033[96m FR: {part.function_response.response}\033[0m ',
          flush=True,
      )
    else:
      print(f'\033[96m {part}\033[0m ', flush=True, end='')